# Python Client for GCS

Google Cloud Storage (GCS), buckets to hold information, data, files.

This notebook illustrates some common interactions with GCS and provides tips on using the Python Client to:
- List buckets
- List files
- create buckets
- delete buckets
- Download files
- Upload files
- more

Resources:
- [Product](https://cloud.google.com/storage)
- [Client API](https://github.com/googleapis/python-storage)
- [Client API Documentation](https://cloud.google.com/python/docs/reference/storage/latest)


---
## Setup

inputs:

In [188]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [189]:
REGION = 'us-central1'
EXPERIMENT = 'gcs'
SERIES = 'tips'

packages:

In [240]:
from google.cloud import storage
import os, shutil
import glob
from datetime import datetime

clients:

In [191]:
gcs = storage.Client()

parameters:

In [192]:
DIR = f'temp/{EXPERIMENT}'
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

environment:
- remote directory named DIR if exists
- create directory DIR
- check for existance of DIR
- list contents of directory one level higher than DIR

In [254]:
shutil.rmtree(DIR, ignore_errors = True)
os.mkdir(DIR)
print(os.path.exists(DIR))
os.listdir(DIR + '/../')

True


['job-parms', 'multi', 'gcs']

---
## Buckets

Buckets are the actual storage locations of files.  They have resources in projects, have properties and permissions, and are located in a region.

### Is there a bucket in this project with the same names as the project?

In [194]:
PROJECT_ID

'statmike-mlops-349915'

In [195]:
lookup = gcs.lookup_bucket(PROJECT_ID)
type(lookup)

google.cloud.storage.bucket.Bucket

In [196]:
lookup = gcs.lookup_bucket(PROJECT_ID+'not_real_name')
type(lookup)

NoneType

### List the buckets in the current project:

In [197]:
list(gcs.list_buckets())

[<Bucket: cloud-ai-platform-a68e7f3a-fac8-47f6-9f92-fff95c09cdb8>,
 <Bucket: statmike-mlops-349915>,
 <Bucket: statmike-mlops-349915-vertex-pipelines-us-central1>]

In [198]:
list(gcs.list_buckets(prefix = 'statmike'))

[<Bucket: statmike-mlops-349915>,
 <Bucket: statmike-mlops-349915-vertex-pipelines-us-central1>]

### Create a new bucket

In [199]:
bucket = gcs.bucket(PROJECT_ID + TIMESTAMP)

In [200]:
bucket = gcs.create_bucket(bucket, project = PROJECT_ID, location = REGION)
bucket

<Bucket: statmike-mlops-34991520220918185010>

In [201]:
list(gcs.list_buckets(prefix = 'statmike'))

[<Bucket: statmike-mlops-349915>,
 <Bucket: statmike-mlops-349915-vertex-pipelines-us-central1>,
 <Bucket: statmike-mlops-34991520220918185010>]

### Retrieve a specific bucket:

In [202]:
bucket = gcs.bucket(PROJECT_ID + TIMESTAMP)

In [203]:
bucket

<Bucket: statmike-mlops-34991520220918185010>

In [204]:
bucket.name

'statmike-mlops-34991520220918185010'

In [205]:
bucket.path

'/b/statmike-mlops-34991520220918185010'

## Files (blobs)

Files are objects, called blobs.  The name includes and prefix you want to use to represent a folder structure.  That's right, there are no actual folders in object storage.  Just files name prefixed with folder like names to help organize and find information.  

### Make some local files

In [206]:
n_folders = 3
n_files = 100

for folder in range(n_folders):
    if not os.path.exists(f'./{DIR}/folder_{folder}'): os.mkdir(f'./{DIR}/folder_{folder}')
    for f in range(n_files):
        with open(f'./{DIR}/folder_{folder}/myfile_{f}.txt', 'w') as file:
            file.write(f'Creating the example file named: myfile_{f}.txt')

In [207]:
os.listdir(f'./{DIR}')[0:10]

['folder_1', 'folder_2', 'folder_0']

In [208]:
os.listdir(f'./{DIR}/folder_0')[0:10]

['myfile_95.txt',
 'myfile_93.txt',
 'myfile_65.txt',
 'myfile_98.txt',
 'myfile_14.txt',
 'myfile_10.txt',
 'myfile_49.txt',
 'myfile_38.txt',
 'myfile_90.txt',
 'myfile_25.txt']

### Uploading Files to Bucket

Get a list of files in the local folder:

In [209]:
glob.glob(f'./{DIR}/**/**')[0:5]

['./temp/gcs/folder_1/myfile_95.txt',
 './temp/gcs/folder_1/myfile_93.txt',
 './temp/gcs/folder_1/myfile_65.txt',
 './temp/gcs/folder_1/myfile_98.txt',
 './temp/gcs/folder_1/myfile_14.txt']

In [210]:
gcs_path_prefix = 'my_folder/my_subfolder/'

In [211]:
for file in glob.glob(f'./{DIR}/**/**'):
    file_path = ('/').join(file.split('/')[-2:]) # just the subfolder and filename
    blob = bucket.blob(path_prefix + file_path)
    blob.upload_from_filename(file)

### List Files in Bucket

In [212]:
list(bucket.list_blobs(max_results = 5))

[<Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_0/myfile_0.txt, 1663527080523988>,
 <Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_0/myfile_1.txt, 1663527076811369>,
 <Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_0/myfile_10.txt, 1663527075714297>,
 <Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_0/myfile_11.txt, 1663527077028392>,
 <Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_0/myfile_12.txt, 1663527076591163>]

In [213]:
list(bucket.list_blobs(max_results = 5, prefix = gcs_path_prefix + 'folder_1/'))

[<Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_1/myfile_0.txt, 1663527055578273>,
 <Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_1/myfile_1.txt, 1663527052134430>,
 <Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_1/myfile_10.txt, 1663527051142485>,
 <Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_1/myfile_11.txt, 1663527052424206>,
 <Blob: statmike-mlops-34991520220918185010, my_folder/my_subfolder/folder_1/myfile_12.txt, 1663527052030295>]

### Downloading

In [214]:
os.makedirs(f'./{DIR}/downloaded')

In [215]:
os.listdir(f'./{DIR}')

['downloaded', 'folder_1', 'folder_2', 'folder_0']

download files from one subfolder on GCS to a local folder:

In [228]:
for blob in bucket.list_blobs(prefix = gcs_path_prefix + 'folder_0/', delimiter = '/'):
    blob.download_to_filename(f"./{DIR}/downloaded/{blob.name.split('/')[-1]}")

In [230]:
os.listdir(f'./{DIR}/downloaded')[0:10]

['myfile_95.txt',
 'myfile_93.txt',
 'myfile_65.txt',
 'myfile_98.txt',
 'myfile_14.txt',
 'myfile_10.txt',
 'myfile_49.txt',
 'myfile_38.txt',
 'myfile_90.txt',
 'myfile_25.txt']

## Buckets, Again

### Delete The Bucket

In [231]:
bucket

<Bucket: statmike-mlops-34991520220918185010>

In [232]:
bucket.delete()
# results in error: Conflict: 409 DELETE https://path: The bucket you tried to delete is not empty.

Conflict: 409 DELETE https://storage.googleapis.com/storage/v1/b/statmike-mlops-34991520220918185010?prettyPrint=false: The bucket you tried to delete is not empty.

In [233]:
bucket.delete(force = True)
# results in error: ValueError: Refusing to delete bucket with more than 256 objects. If you actually want to delete this bucket, please delete the objects yourself before calling Bucket.delete().

ValueError: Refusing to delete bucket with more than 256 objects. If you actually want to delete this bucket, please delete the objects yourself before calling Bucket.delete().

In [234]:
bucket.delete_blobs(blobs = list(bucket.list_blobs()))

In [235]:
list(gcs.list_buckets())

[<Bucket: cloud-ai-platform-a68e7f3a-fac8-47f6-9f92-fff95c09cdb8>,
 <Bucket: statmike-mlops-349915>,
 <Bucket: statmike-mlops-349915-vertex-pipelines-us-central1>,
 <Bucket: statmike-mlops-34991520220918185010>]

In [236]:
bucket.delete()
# works, because the bucket is empty

In [237]:
list(gcs.list_buckets())

[<Bucket: cloud-ai-platform-a68e7f3a-fac8-47f6-9f92-fff95c09cdb8>,
 <Bucket: statmike-mlops-349915>,
 <Bucket: statmike-mlops-349915-vertex-pipelines-us-central1>]